### Rozszerzanie typów za pomocą osadzania funkcji typów danych jako metod klasy:

In [1]:
class Set:
    def __init__(self, value =[]):
        self.data = []
        self.concat(value)
        
    def intersect(self, other):
        res = []
        for x in self.data:
            if x in other:                  # wybor wspolnych elementow
                res.append(x)
        return Set(res)
            
    def union(self, other):
        res = self.data[:]
        for x in other:
            if not x in res:              # dodanie nowych elementów z other do res
                res.append(x)
        return Set(res)
            
    def concat(self, value):
        for x in value:
            if not x in self.data:       # usuwanie duplikatów z listy
                self.data.append(x)
            
    def __len__(self):
        return len(self.data)
    def __getitem__(self, key):
        return self.data[key]
    def __and__(self, other):
        return self.intersect(other)
    def __or__(self, other):
        return self.union(other)
    def __repr__(self):
        return 'Zbiór: ' + repr(self.data)

In [2]:
x = Set([1, 3, 5, 7])

In [3]:
print(x.union(Set([1,4,7])))

Zbiór: [1, 3, 5, 7, 4]


In [4]:
print(x | Set([1,4,6]))

Zbiór: [1, 3, 5, 7, 4, 6]


### Rozszerzanie typów za pomocą klas podrzędnych: można zmienić zachowanie jakichs obiektów, np krotek, list, słowników itp budując klasę, która będzie podklasą tego obiektu, np:
        class MyList(list)
        class MyTulpe(tuple)
        class MyString(string)

### w nawiasie wpisujemy nazwę typu danych, tak jak przy tworzeniu zwykłej klasy podrzędnej.

In [5]:
class MyList(list):   # klasa podrzędna klasy obiektu 'lista'
    def __getitem__(self, offset):
        print(f'indeksowanie {self} w pozycji {offset}:')    
        return list.__getitem__(self, offset - 1)    # indeksowanie bedzie zaczynało się od pozycji 1 a nie 0

if __name__=='__main__':
    print(list('abc'))
    x = MyList('abc')
    print(x)
    
    print(x[1])
    print(x[3])
    
    x.append('fff')
    print(x)
    x.reverse()
    print(x)

['a', 'b', 'c']
['a', 'b', 'c']
indeksowanie ['a', 'b', 'c'] w pozycji 1:
a
indeksowanie ['a', 'b', 'c'] w pozycji 3:
c
['a', 'b', 'c', 'fff']
['fff', 'c', 'b', 'a']


In [6]:
# ten sam przyklad co na poczatku, ale wykorzystujące nadklasę "list", dzieki temu nie trzeba definiowac funkcji ktore są typowe dla list
class SetList(list):
    def __init__(self, value =[]):   
#        list.__init__([])          # !!! nie wiem po co on dał tą linijkę, po usunięciu jej klasa działa tak samo !!!
        self.concat(value)           # odpalam metodę concat, która kopiuje zmienne wartosci domyslne
    
    def intersect(self, other):
        res = []
        for x in self:
            if x in other:          # wybor wspolnych elementow
                res.append(x)
        return SetList(res)
            
    def union(self, other):
        res = SetList(self)
        res.concat(other)
        return res    
        
    def concat(self, value):
        for x in value:
            if not x in self:       # usuwanie duplikatów
                self.append(x)
            
    def __and__(self, other):
        return self.intersect(other)
    
    def __or__(self, other):
        return self.union(other)
    
    def __repr__(self):
        return 'Zbiór: ' + list.__repr__(self)

In [7]:
X = SetList([1,3,5,7])

In [8]:
Y = SetList([2,1,4,5,6,4])

In [9]:
print(X, Y,'len: ', len(X))   # funkcja len dziala bez definiowania, bo python wie, ze klasa SetList to lista

Zbiór: [1, 3, 5, 7] Zbiór: [2, 1, 4, 5, 6] len:  4


In [10]:
print(X.intersect(Y), Y.union(X))

Zbiór: [1, 5] Zbiór: [2, 1, 4, 5, 6, 3, 7]


In [11]:
print(X & Y, Y | X)

Zbiór: [1, 5] Zbiór: [2, 1, 4, 5, 6, 3, 7]


In [12]:
X.reverse()

In [13]:
print(X)

Zbiór: [7, 5, 3, 1]


In [14]:
class C: pass
class D: pass

In [15]:
c = C()

In [16]:
d = D()

In [17]:
type(c)

__main__.C

In [18]:
type(d)

__main__.D

In [19]:
type(c) == type(d)

False

In [20]:
c.__class__, d.__class__

(__main__.C, __main__.D)

In [21]:
c1, c2 = C(), C()

In [22]:
type(c1)==type(c2)

True

### KOLEJNOŚĆ PRZESZUKIWANIA DRZEWA KLAS (od dołu do góry i od lewej do prawej: najpierw poszukuje wszerz w klasach nadrzędnych pierwszego poziomu, od lewej do prawej, a następnie przechodzi piętro wyżej.) i dziedziczenie diamentowe (gdy kilka klas nadrzednych ma jednego przodka):

In [23]:
class A:
    attr = 'a'

In [24]:
class B(A): pass

In [25]:
class C(A): 
    attr = 'c(a)'

In [26]:
class E(B,C): pass

In [27]:
e = E()

In [28]:
e.attr  # tutaj wyszukiwanie - najpierw E, potem B, potem C (C.attr przyszlonił A.attr)

'c(a)'

In [29]:
e.__dict__

{}

In [30]:
class D(B,C):
    attr = B.attr  # zmuszenie pythona do przeszukania attr w klasie B, wiec znajduje ją wyzej, w klasie A

In [31]:
x = D()

In [32]:
x.attr

'a'

In [33]:
# to samo dotyczy dziedziczenia metod
class A:
    def m(self): print('A')

In [34]:
class C(A):
    def m(self): print('C(A)')

In [35]:
class B(A): pass

In [36]:
class D(B,C): pass

In [37]:
d = D()

In [38]:
d.m()

C(A)


In [39]:
class D(B,C): m = B.m

In [40]:
ddd = D()

In [41]:
ddd.m()

A


In [42]:
ddd.k = 9

In [43]:
ddd.__dict__  # __dict__ wywoluje atrybuty instancji (czyli self.atrybuty)

{'k': 9}

### Sloty w klasach: atrybut  __ slots __   ogranicza ilość dostępnych nazw, które można wykorzystać w  instancjach, do tych wymienionych w liscie *__ slots __ = [...]*

### Instancja ma bezposredni dostęp do slotów klasy *na najnizszym poziomie dziedziczenia.*

In [44]:
class limiter:
    __slots__ = ['age', 'name', 'job']  # instancje będą mogły mieć tylko te trzy atrybuty
    atrybut = 5    # atrybuty klas dzialaja bez zmian 

In [45]:
x = limiter()

In [46]:
x.age = 40

In [47]:
# x.ape = 4   # nie mozna utworzyc atrybutu niewymienionego w __slots__; eliminacja problemu z literowkami

In [48]:
#x.__dict__  # klasy w ktorych zdefiniowano __slots__ tracą atrybut __dict__

In [49]:
getattr(x, 'age')  # skoro x.__dict__ nie dziala, trzeba dobierac sie do atrybutow w inny sposob

40

In [50]:
'age' in dir(x)

True

In [51]:
dir(x)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 'age',
 'atrybut',
 'job',
 'name']

In [52]:
# problem rozwiazuje dopisanie __dict__ do listy __slots__
class limiter2:
    __slots__ = ['age', 'name', 'job', '__dict__'] 
    atrybut = 5   
    def __init__(self): self.d = 4      # d zostaje zapisane w __dict__, wiec bedzie widoczne w instancjach po wywolaniu __dict__, mimo ze te atrybuty sa dziedziczone po klasie

In [53]:
y=limiter2()

In [54]:
y.name = 'andżej'

In [55]:
y.__slots__

['age', 'name', 'job', '__dict__']

In [56]:
y.__dict__  

{'d': 4}

In [57]:
y.mm = 8   # mozna znowu dopisywac atrybuty, lądują w __dict__, mimo ze jest __slots__

In [58]:
y.mm

8

In [59]:
getattr(y, 'mm')

8

In [60]:
y.__dict__  

{'d': 4, 'mm': 8}

### Wiele slotów w klasach nadrzędnych:
### • Jeśli klasa dziedziczy po klasie nieposiadającej atrybutu __ slots __, atrybut __ slots __ klasy nadrzędnej będzie zawsze dostępny, przez co atrybut __ slots __ w klasie podrzędnej nie będzie miał zastosowania.
### • Jeśli klasa definiuje te same nazwy slotów co klasa nadrzędna, to sloty klasy nadrzędnej będą dostępne poprzez deskryptor klasy.
### • Deklaracja __ slots __ dotyczy wyłącznie klasy, w której występuje, klasy podrzędne będą posiadały __ dict __, chyba że również definiują __ slots __.

In [61]:
class E:
    __slots__ = ['c', 'd']

In [62]:
class D(E):
    __slots__ = ['a', '__dict__']

In [63]:
X = D()

In [64]:
X.a = 1; X.b = 2; X.c = 3

In [65]:
X.a

1

In [66]:
X.b

2

In [67]:
X.c

3

In [68]:
X.__dict__

{'b': 2}

In [69]:
X.__slots__   # X dziedziczy sloty tylko po D

['a', '__dict__']

In [70]:
D.__slots__

['a', '__dict__']

In [71]:
E.__slots__   # X nie dziedziczy tych slotów

['c', 'd']

In [72]:
for attr in list(getattr(X, '__dict__', [])) + getattr(X, '__slots__', []):
    print(attr, '=>', getattr(X, attr))           # atrybut X.c nie został odnaleziony ani w __dict__ ani w __slots__

b => 2
a => 1
__dict__ => {'b': 2}


### Funkcja wbudowana *property* - zawiera trzy metody (get, set, delete) + dokumentacja. Jeśli jeden z tych argumentów zostanie pominięty albo przekazany jako none, to ta operacja nie jest obsługiwana. Zazwyczaj zapisywane na najwyzszym poziomie instrukcji class. Dostępy do atrybutów klasy są automatycznie przekierowywane do jednej z metod akcesorów przekazanej do funkcji property.

### Składnia:
    class Klasa:
    def getKlasa(self):
        return ...
    def setKlasa(self, value):
        print('tekst: ', value)
        self.data = value 
    def delKlasa(self):
        del objekt
    def docKlasa(self):
        print('Dokumentacja')
    age = property(getKlasa, setKlasa, delKlasa, docKlasa) # Operacje get, set, del, dokumentacja

In [73]:
# starymi metodami __getattr__ i __setattr__ (przed erą funkcji property):
class classic:
    
    def __getattr__(self,name):
        if name == 'age':
            return 40
        else:
            raise AttributeError
    
    def __setattr__(self, name, value):
        print('ustawienie:', name, value)
        if name == 'age':
            self.__dict__['_age'] = value
        else:
            self.__dict__[name] = value

In [74]:
x = classic()

In [75]:
x.age   # Wykonuje __getattr__

40

In [76]:
# x.name   # Wykonuje __getattr__, wywala AttributeError

In [77]:
x.age = 42

ustawienie: age 42


In [78]:
x.age  # wykonuje __getattr__

40

In [79]:
x._age

42

In [80]:
x.job = 'diler'   # wykonuje __setattr__

ustawienie: job diler


In [81]:
x.job    # normalne wywołanie

'diler'

In [82]:
class newprops:
    def getage(self):
        return 40
    def setage(self, value):
        print('ustawienie wieku: ', value)
        self._age = value # _X to wewnetrzna nazwa w klasie wg konwencji (nie jest to element składni)
    age = property(getage, setage, None, None) # Operacje get, set, del, dokumentacja

In [83]:
y = newprops()

In [84]:
y.age   # wykonuje getage

40

In [85]:
# y.name   # normalne pobranie; wywala AttributeError

In [86]:
y.age = 42   # wykonuje setage

ustawienie wieku:  42


In [87]:
y.age    # wykonuje getage

40

In [88]:
y._age   # wykonuje normalne pobranie

42

In [89]:
y.job = 'diler'   # normalne przypisanie

In [90]:
y.job   # normalne pobranie

'diler'

### Typy metod:
### 1. Metody instancji - czyli te standardowe, z argumentem *self*
### 2. Metody statyczne - niewiążące się z instancją, nie posiadają argumentu *self*, nie obsługują żadnej instancji tylko klasę. Można je wywołać z poziomu klasy.
### 3. Metody klasy - w argumencie otrzymuje klasę a nie instancje

In [91]:
# przyklad zastosowania metody statycznej: zliczanie instancji powstałych na bazie ponizszej klasy:
class Licznik:
    numInstances = 0
    def __init__(self):
        Licznik.numInstances = Licznik.numInstances + 1
    def printNumInst():       # metoda statyczna - nie ma argumentu 'self', wiec dane przechowuje w klasie 
        print('Liczba utworzonych instancji: ', Licznik.numInstances)

In [92]:
a = Licznik()

In [93]:
b = Licznik()

In [94]:
c = Licznik()

In [95]:
Licznik.printNumInst()

Liczba utworzonych instancji:  3


In [96]:
# c.printNumInst()  # nie zadziala, bo przekazuje argument "c" do metody ktora nie czeka na argument 'self'

In [97]:
# jako alternatywe dla metod statycznych, mozna uzyc definicji funkcji poza klasą:

def printNumInstances():
    print("Liczba utworzonych instancji: ", Licznik2.numInstances)

In [98]:
class Licznik2:
    numInstances = 0
    def __init__(self):
        Licznik2.numInstances = Licznik2.numInstances + 1

In [99]:
a = Licznik2()

In [100]:
b = Licznik2()

In [101]:
c = Licznik2()

In [102]:
printNumInstances()

Liczba utworzonych instancji:  3


In [103]:
Licznik2.numInstances

3

In [104]:
# kolejny sposób, niestety do sprawdzenia liczby instancji, potrzebna jest przynajmniej jedna instancja
class Licznik3:
    numInstances = 0
    def __init__(self):
        Licznik3.numInstances = Licznik3.numInstances + 1
    def printNumInstances(self):
        print("Liczba utworzonych instancji: ", Licznik3.numInstances)

In [105]:
x, y, z = Licznik3(), Licznik3(), Licznik3()

In [106]:
#Licznik3.printNumInstances() # nie działa, bo czeka na self

In [107]:
x.printNumInstances()

Liczba utworzonych instancji:  3


In [108]:
x.printNumInstances()

Liczba utworzonych instancji:  3


In [109]:
Licznik3().printNumInstances()   # uzycie tej funkcji modufikuje licznik!

Liczba utworzonych instancji:  4


#### METODY KLAS:

In [110]:
class Methods:
    def instancemet(self, x):          # Zwykła metoda instancji: otrzymuje self
        print(self, x)
    def staticmet(x):                  # Metoda statyczna: instancja nie jest przekazywana
        print(x)
    def classicmet(cls, x):            # Metoda klasy: otrzymuje klasę, nie instancję
        print(cls, x)
#    staticmet = staticmethod(staticmet)    # Przekształcenie staticmet w metodę statyczną - nieobowiazkowe od pythona 3.0
    classicmet = classmethod(classicmet)   # Przekształcenie classicmet w metodę klasy

In [111]:
obj = Methods()

In [112]:
obj.instancemet(1)

<__main__.Methods object at 0x0000000004FF9580> 1


In [113]:
Methods.instancemet(obj,2)

<__main__.Methods object at 0x0000000004FF9580> 2


In [114]:
Methods.staticmet(4)

4


In [115]:
Methods.classicmet(5)

<class '__main__.Methods'> 5


In [116]:
# Zliczanie instancji z użyciem metod statycznych:
class Licznik4:
    numInst = 0
    def __init__(self):
        Licznik4.numInst += 1
    def printNumInst():
        print('Liczba instancji:', Licznik4.numInst)
    printNumInst = staticmethod(printNumInst)  # zapis powoduje ze nazwa metody pozostala lokalna w klasie, nie bedzie powodowac konfliktow z innymi nazwami np w module

In [117]:
a = Licznik4()

In [118]:
b = Licznik4()

In [119]:
c = Licznik4()

In [120]:
Licznik4.printNumInst()

Liczba instancji: 3


In [121]:
a.printNumInst()

Liczba instancji: 3


In [122]:
class SubLicznik(Licznik4):
    def printNumInst():    # nadpisanie metody statycznej z Licznik4
        print("cos ekstra... ;)")
        Licznik4.printNumInst()    # wywolanie oryginalnej metody statycznej z klasy nadrzednej
    printNumInst = staticmethod(printNumInst)

In [123]:
x = SubLicznik()

In [124]:
x.printNumInst()

cos ekstra... ;)
Liczba instancji: 4


In [125]:
SubLicznik.printNumInst()  # zlicza razem z instancjami klasy nadrzednej

cos ekstra... ;)
Liczba instancji: 4


In [126]:
# Zliczanie instancji z użyciem metod klas:
class Licznik5:
    numInst = 0
    def __init__(self):
        Licznik5.numInst += 1
    def printNumInst(cls):
        print('liczba instancji: ', cls.numInst, cls)
    printNumInst = classmethod(printNumInst)

In [127]:
a,b = Licznik5(), Licznik5()

In [128]:
a.printNumInst()

liczba instancji:  2 <class '__main__.Licznik5'>


In [129]:
class SubLicznik5(Licznik5):
    def printNumInst(cls):
        print('cos ekstra :)', cls)
        Licznik5.printNumInst()
    printNumInst = classmethod(printNumInst)

In [130]:
x, y = Licznik5(), SubLicznik5()

In [131]:
x.printNumInst()

liczba instancji:  4 <class '__main__.Licznik5'>


In [132]:
y.printNumInst()   # zlicza tylko wszystkie instancji klasy nadrzednej i podrzednej

cos ekstra :) <class '__main__.SubLicznik5'>
liczba instancji:  4 <class '__main__.Licznik5'>


In [133]:
#Zliczanie instancji dla każdej z klas z użyciem metod klas:
class Licznik6:
    numInst = 0 
    def count(cls):
        cls.numInst += 1
    def __init__(self):
        self.count()
    count = classmethod(count)

In [134]:
class SubLicznik6(Licznik6):
    numInst = 0 
    def __init__(self):
        Licznik6.__init__(self)

In [135]:
class SubPass(Licznik6): 
    numInst = 0

In [136]:
x = Licznik6()

In [137]:
y1, y2 = SubLicznik6(), SubLicznik6()

In [138]:
z1, z2, z3 = SubPass(), SubPass(), SubPass()

In [139]:
x.numInst, y1.numInst, z1.numInst  # oddzielne zliczanie instancji każdej klasy

(1, 2, 3)

### Dekoratory funkcji: zapisywany przed definicja funkcji
składnia:  

    class C:
        @staticmethod
        def meth():
            ...
            
            
w tym wypadku jest to równoznaczne z zapisem:

    class C:
        def meth():
            ...
        meth = staticmethod(meth)

In [140]:
class Licznik7:
    numInstances = 0
    def __init__(self):
        Licznik7.numInstances = Licznik7.numInstances + 1
    @staticmethod
    def printNumInstances():
        print("Liczba utworzonych instancji: ", Licznik7.numInstances)

In [141]:
a = Licznik7()

In [142]:
b = Licznik7()
c = Licznik7()

In [143]:
Licznik7.numInstances

3

In [144]:
Licznik7.printNumInstances()

Liczba utworzonych instancji:  3


In [145]:
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func  = func
    def __call__(self, *args):
        self.calls += 1
        print(f'wywołanie nr {self.calls} do {self.func.__name__}')
        self.func(*args)
@tracer                       # Równoważne wywołaniu spam = tracer(spam)
def spam(a, b, c):            # Opakowanie funkcji spam w obiekt dekoratora
    print(a, b, c)

In [146]:
spam(1, 2, 3)   # ponieważ spam = tracer(spam), po wywolaniu uruchamia sie __call__ klasy tracer

wywołanie nr 1 do spam
1 2 3


In [147]:
spam('a', 'b', 'c')

wywołanie nr 2 do spam
a b c


In [148]:
spam(4, 5, 6) 

wywołanie nr 3 do spam
4 5 6


### Dekoratory Klas: 
składnia:

    def decorator(aClass):
        ...
    @decorator
    class C:
        ...
jest to równoznaczne z zapisem:

    def decorator(aClass):
        ...
    class C:
        ...
    C = decorator(C)

In [149]:
def count(aClass):
    aClass.numInstances = 0
    return aClass                     # Zwracamy samą klasę, nieobiekt opakowujący
@count
class Licznik: ...                    # Równoważne wywołaniu Licznik = count(Licznik)
@count
class Sub(Licznik): ...               # numInst = 0 nie jest potrzebne
@count
class Other(Licznik): ...

### Pułapki związane z klasami:
### 1. Modyfikacja atrybutów klas może mieć efekty uboczne
### 2. Modyfikowanie mutowalnych obiektów w klasach może mieć efekty uboczne
### 3. Dziedziczenie wielokrotne — kolejność ma znaczenie
### 4. Przesadne opakowywanie - zbyt wiele poziomow dziedziczenia = zagmatwany kod

In [150]:
# ad 1. Modyfikacja atrybutów klas może mieć efekty uboczne
class X:
    a = 1

In [151]:
I = X()

In [152]:
I.a

1

In [153]:
X.a

1

In [154]:
X.a = 2

In [155]:
I.a   # zmienna a w klasie X została zmodyfikowana, wiec w instancjach tez juz jest zmieniona

2

In [156]:
# ad 2. Modyfikowanie mutowalnych obiektów w klasach może mieć efekty uboczne
class C:
    obiektmutowalny = []
    def __init__(self):
        self.obj = []

In [157]:
x = C()

In [158]:
y = C()

In [159]:
y.obiektmutowalny, x.obj

([], [])

In [160]:
x.obiektmutowalny.append('cos')
x.obj.append('cos wiecej')

In [161]:
x.obiektmutowalny, x.obj

(['cos'], ['cos wiecej'])

In [162]:
y.obiektmutowalny, y.obj  # UWAGA! zmienil sie obiektmutowalny w KLASIE C!, a zatem tez w instancji y

(['cos'], [])

In [163]:
C.obiektmutowalny

['cos']